<a href="https://colab.research.google.com/github/achiral/mosaic_art/blob/main/mosaic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### preparation

In [1]:
# google driveからcolaboにファイルをコピーするメソッド
import requests

def download_file_from_google_drive(id, destination):
    print('---- download_file_from_google_drive start ----')
    URL = "https://docs.google.com/uc?export=download"

    try:
        session = requests.Session()

        response = session.get(URL, params = { 'id' : id }, stream = True)
        print('response.status_code: {}'.format(response.status_code))
        print('response.headers: {}'.format(response.headers))

        token = get_confirm_token(response)
        print('token'.format(token))

        if token:
            print(token)
            params = { 'id' : id, 'confirm' : token }
            response = session.get(URL, params = params, stream = True)

        save_response_content(response, destination)
    except Exception as e:
        print(e)


def get_confirm_token(response):
    for key, value in response.cookies.items():
        print('key: {}'.format(key))
        print('value: {}'.format(value))
       
        if key.startswith('download_warning'):
            return value
            
        if key == 'NID':
            print('NID')
            token = value.split('=')
            print(token)
            return token[1]

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
# 実行環境のrootディレクトリの確認
# %cd /content/
# !ls -lah

In [2]:
# Google driveマウント
from google.colab import drive
drive.mount('/content/gdrive')

# マウントしたGoogle Drive情報の確認
!ls gdrive -lah

Mounted at /content/gdrive
total 16K
dr-x------  2 root root 4.0K Dec 21 08:36 .file-revisions-by-id
drwx------ 10 root root 4.0K Dec 21 08:36 MyDrive
dr-x------  2 root root 4.0K Dec 21 08:36 .shortcut-targets-by-id
drwx------  5 root root 4.0K Dec 21 08:36 .Trash-0


In [3]:
# datasetを配置するディレクトリを作成
!mkdir ./dataset
!mkdir /content/output

In [ ]:
# %cd /content/gdrive/MyDrive/'Colab Notebooks'/lightweight-gan/results
# # フォルダを zip 圧縮する
# !zip -r ./inner-generated-100.zip ./inner-generated-100


# # 圧縮した zip ファイルをダウンロードする
# # from google.colab import files
# # files.download("./inner-generated-100.zip")

# %cd /content
# %pwd

### 画像の準備

In [12]:
# dataset(zipでgdriveに格納)をColabにコピー
# datasetのサイズによって時間がかかる
%%time
# !cp "./gdrive/MyDrive/dataset/endless_summer.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/amime_128_mini.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/inner_512.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/inner2_512.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/inner3_512.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/inner3_2048.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/inner3_2048alpha.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/face1024.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/skin_512.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/skin_2048.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/onepiece_512.zip" "./dataset"
# !cp "/content/gdrive/MyDrive/Colab Notebooks/lightweight-gan/results/inner-generated-100.zip" "./dataset"
# !cp "./gdrive/MyDrive/dataset/another_anime_face_256.zip" "./dataset"
!cp "./gdrive/MyDrive/dataset/blocks.zip" "./dataset"

# 目的(target)の画像をコピー
!cp "./gdrive/MyDrive/dataset/target.jpg" "./dataset"
!cp "./gdrive/MyDrive/dataset/target2.png" "./dataset"
!cp "./gdrive/MyDrive/dataset/target3.png" "./dataset"

# %cd /content
# %pwd

CPU times: user 21.1 ms, sys: 34.5 ms, total: 55.6 ms
Wall time: 1.07 s


### 画像のコピーと解凍

In [13]:
# コピー(またはダウンロード)したdataset解凍
%%time
# !unzip ./dataset/endless_summer.zip -d ./dataset
# !unzip ./dataset/amime_128_mini.zip -d ./dataset
# !unzip ./dataset/inner_512.zip -d ./dataset
# !unzip ./dataset/inner2_512.zip -d ./dataset
# !unzip ./dataset/inner3_512.zip -d ./dataset
# !unzip ./dataset/inner3_2048.zip -d ./dataset
# !unzip ./dataset/inner3_2048alpha.zip -d ./dataset
# !unzip ./dataset/face1024.zip -d ./dataset
# !unzip ./dataset/skin_512.zip -d ./dataset
# !unzip ./dataset/skin_2048.zip -d ./dataset
# !unzip ./dataset/onepiece_512.zip -d ./dataset
# !unzip ./dataset/inner-generated-100.zip -d ./dataset
# !unzip ./dataset/another_anime_face_256.zip -d ./dataset
!unzip ./dataset/blocks.zip -d ./dataset


# データ件数の確認
!find dataset/* -type f | wc -l

Archive:  ./dataset/blocks.zip
replace ./dataset/__MACOSX/._blocks? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./dataset/blocks/weathered_cut_copper.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./dataset/__MACOSX/blocks/._weathered_cut_copper.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./dataset/blocks/glass_blue.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: none
replace ./dataset/__MACOSX/blocks/._glass_blue.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
1927
CPU times: user 490 ms, sys: 80.2 ms, total: 570 ms
Wall time: 1min 12s


In [14]:
# 作業ディレクトリの移動
# %cd /content/gdrive/MyDrive/'Colab Notebooks'/lightweight-gan
%pwd
%ls -al

total 28
drwxr-xr-x 1 root root 4096 Dec 21 08:36 ./
drwxr-xr-x 1 root root 4096 Dec 21 08:35 ../
drwxr-xr-x 4 root root 4096 Dec 19 14:30 .config/
drwxr-xr-x 4 root root 4096 Dec 21 08:39 dataset/
drwx------ 5 root root 4096 Dec 21 08:36 gdrive/
drwxr-xr-x 2 root root 4096 Dec 21 08:36 output/
drwxr-xr-x 1 root root 4096 Dec 19 14:31 sample_data/


#mosaic art #1

In [22]:
%cd /content
%pwd

/content


'/content'

In [9]:
import numpy as np
import cv2
import glob

In [40]:
# IMG_PATH = './images/*.png'          # モザイク素材画像のパス
IMG_PATH = R"./dataset/blocks/*.png"  # モザイク素材画像のパス
# INPUT_PATH = R"./dataset/target.jpg"  # モザイク写真にしたい画像のパス
# INPUT_PATH = R"./dataset/target.png"  # モザイク写真にしたい画像のパス
# INPUT_PATH = R"./dataset/target1.jpg"  # モザイク写真にしたい画像のパス
# INPUT_PATH = R"./dataset/target1.png"  # モザイク写真にしたい画像のパス
# INPUT_PATH = R"./dataset/target2.jpg"  # モザイク写真にしたい画像のパス
# INPUT_PATH = R"./dataset/target2.png"  # モザイク写真にしたい画像のパス
# INPUT_PATH = R"./dataset/target3.jpg"  # モザイク写真にしたい画像のパス
INPUT_PATH = R"./dataset/target3.png"  # モザイク写真にしたい画像のパス
MULTIPLY = 1                          # モザイク写真にしたい画像拡大

In [ ]:
# files = glob.glob('./cats/*.jpg')
files = glob.glob(IMG_PATH)
size = 50    #150×150の画像を50×50にresizeして行列で表す
 
small_img = []
 
for file in files:
    image = cv2.imread(file)
    image = cv2.resize(image, (size, size))
    image = image /255
    small_img.append(image)
 
small_img = np.asarray(small_img)
pic_filename = INPUT_PATH                 #モザイク写真にしたい画像のパス
 
big_img = cv2.imread(pic_filename)         #big_imgのサイズが50で割り切れるように指定する。このサイズを大きくするほど鮮明なモザイク写真ができる
height = (big_img.shape[0] - big_img.shape[0] % size) * 10 * MULTIPLY
width = (big_img.shape[1] - big_img.shape[1] % size) * 10 * MULTIPLY
 
big_img = big_img / 255
big_img = np.asarray(big_img)
big_img = cv2.resize(big_img, (width, height))
 
for i in range(int(height / size)):
    print('%d / %d' %(i + 1, int(height / size)))
 
    for j in range(int(width / size)):
        cut = big_img[size * i: size * (i + 1), size * j: size * (j + 1)]
        losses = [] 
        
        #small_imgの中で小さく分けた領域に一番似ているものを探す
        for s_img in small_img:
            loss = cut - s_img
            loss = loss * loss
            loss = np.sum(loss)
            losses.append(loss) 
     
        losses = np.asarray(losses)
        big_img[size * i: size * (i + 1), size * j: size * (j + 1)] = small_img[np.argmin(losses)]
 
big_img = (big_img * 255).astype(np.int64)
cv2.imwrite('mosaic.png', big_img)

1 / 200
2 / 200
3 / 200
4 / 200
5 / 200
6 / 200
7 / 200
8 / 200
9 / 200
10 / 200
11 / 200
12 / 200
13 / 200
14 / 200
15 / 200
16 / 200
17 / 200
18 / 200
19 / 200
20 / 200
21 / 200
22 / 200
23 / 200
24 / 200
25 / 200
26 / 200
27 / 200
28 / 200
29 / 200
30 / 200
31 / 200
32 / 200
33 / 200
34 / 200
35 / 200
36 / 200
37 / 200
38 / 200
39 / 200
40 / 200
41 / 200
42 / 200
43 / 200
44 / 200
45 / 200
46 / 200
47 / 200
48 / 200
49 / 200
50 / 200
51 / 200
52 / 200
53 / 200
54 / 200
55 / 200
56 / 200
57 / 200
58 / 200
59 / 200
60 / 200


### colab2drive

In [46]:
# %cd /content
# %pwd

In [47]:
## 圧縮してコピー
# !zip -r mosaic.zip mosaic.png 
# !cp logs/mosaic.zip /content/drive/My\ Drive/mosaic.zip

## 圧縮せずコピー
!cp "mosaic.png" "./gdrive/MyDrive/dataset/mosaic.png"
print("----- finished copy -----")

----- finished copy -----


#mosaic art  
don't work

In [27]:
# coding:utf-8
from PIL import Image,ImageFilter
import subprocess
import os
import random
import math
import time
import numpy as np
import glob
import cv2

In [28]:
SourceDir = R"./dataset/blocks"
GoalImage = R"./dataset/target.jpg"
OutputImage="output.jpg"
print(SourceDir)
print(GoalImage)
print(OutputImage)

./dataset/blocks
./dataset/target.jpg
output.jpg


In [29]:
SourceImageSize=(16,16)  # (60,40)

In [30]:
TargetZoom=11

In [31]:
used_count = 3

In [33]:
#==============設定===================
#素材画像の入っているフォルダ
SourceDir = R"./dataset/blocks"

#目標となる画像
GoalImage = R"./dataset/target.jpg"

#出力の名前
OutputImage="output.jpg"

#素材の画像のサイズ(読み込み時にここで指定したサイズに変換される)
SourceImageSize=(10,10)  # (60,40)

#ターゲット画像の倍率
TargetZoom=0.01

# 画像使用制限
used_count = 100000000
#=====================================

# 目標画像のタイル化
# org_image : PILで.open()した画像
# height : 素材画像の縦サイズ
# width : 素材画像の横サイズ
#
# return : RectList タイル
def __create_tile(org_image,height,width):

    # PPIL.Image.open()で読み込んだ画像データを渡すと
    # 形状shapeが(行（高さ）, 列（幅）, 色（チャンネル）)の三次元の配列ndarrayが得られる
    w, h = org_image.size
    RectList=[]
    for y in range(0, math.floor(h / height) + 1):
        for x in range(0, math.floor(w / width) + 1):
            height2 = y * height
            width2 = x * width
            crap_img = org_image.crop((width2, height2, width2 + width, height2 + height))
            # crap_img.show()
            RectList.append((crap_img,width2,height2))
    return RectList

# ヒストグラムを計算する
# image : 画像ファイル
# return : hist_vec ヒストグラムのベクトル
def __clac_hist(img):

    hist_list = []
    color = ['r','g','b']
    images = np.asarray(img)
    # カラーヒストグラムはRGBそれぞれ計算する
    # そのため、3回ループしている
    for i in enumerate(color):
        hist_list.append(cv2.calcHist([images],[i[0]],None,[256],[0,256]))
    return hist_list

if __name__=="__main__":

    startTime=time.time()

    # 目標画像を開きリサイズ
    target=Image.open(GoalImage)
    target=target.resize(tuple(math.floor(i*TargetZoom) for i in target.size))

    # 動画撮影
    # fourcc2=cv2.VideoWriter_fourcc(*fourcc)
    # v=cv2.VideoWriter(OutputMovie,int(fourcc2),Framerate,target.size)

    # 元画像のタイル化
    print("%s:タイルリストを生成開始"%(str(time.time()-startTime)))
    tiles = __create_tile(target,SourceImageSize[1],SourceImageSize[0])
    print("%s:タイルリストを生成完了"%(str(time.time()-startTime)))

    # 素材画像ファイル名を取得
    # 全素材画像をリサイズする
    file_paths = glob.glob(SourceDir+"\\*")
    src_hist_dict = {}
    print("%s:全素材画像ヒストグラムを計算開始"%(str(time.time()-startTime)))
    for file_path in file_paths:
        file_name = file_path.split("\\")[-1]

        # print(file_name)

        # 画像のopenでエラーの場合は次の画像へ
        try:
            src_image =  Image.open(file_path).resize(SourceImageSize)
        except Exception:
            continue
        # src_image.show()
        src_hist_dict[file_name] = [src_image,__clac_hist(src_image),0]
    print("%s:全素材画像ヒストグラムを計算完了"%(str(time.time()-startTime)))

    # カラーヒストグラム計算 素材画像

    Target=Image.new("RGB",target.size,255)

    # タイルの数だけループ
    print("%s:モザイクアート生成開始"%(str(time.time()-startTime)))
    while(len(tiles) > 0):
        result = []

        print("残りタイル：" + str(len(tiles)))
        r=random.randrange(len(tiles)) # タイルの中からランダムで1つ選択
        tileRect=tiles[r]
        del tiles[r]
        # タイルのカラーヒスとグラムを取得する。
        tile_hist = __clac_hist(tileRect[0])
        # RGBを同じベクトルにする
        tile_hist = np.array(tile_hist)
        tile_hist = tile_hist.reshape(tile_hist.shape[0] * tile_hist.shape[1], 1)

        # 元画像のタイルと素材のヒストグラムを比較
        for file_name,src_hist in src_hist_dict.items():
            src_hist = np.array(src_hist[1])
            src_hist = src_hist.reshape(src_hist.shape[0] * src_hist.shape[1], 1)

            d = cv2.compareHist(tile_hist, src_hist, cv2.HISTCMP_INTERSECT)
            result.append([d,file_name])
        result.sort(reverse=True) # ソート

        # ファイルの置き換え
        Target.paste(src_hist_dict[result[0][1]][0],(tileRect[1],tileRect[2]))

        # 画像使用カウントアップ
        src_hist_dict[result[0][1]][2] = src_hist_dict[result[0][1]][2] + 1

        # カウントが一定値に達したら
        if src_hist_dict[result[0][1]][2] == used_count:
            del src_hist_dict[result[0][1]]

        # print("ファイル名 : " + result[0][1])
        # print("x座標 : " + str(tileRect[1]))
        # print("y座標 : " + str(tileRect[2]))

        # Target.show()

    print("%s:モザイクアート生成完了"%(str(time.time()-startTime)))
    Target.save(OutputImage)
    # v.release()

0.4345431327819824:タイルリストを生成開始
0.43557167053222656:タイルリストを生成完了
0.4374699592590332:全素材画像ヒストグラムを計算開始
0.4375312328338623:全素材画像ヒストグラムを計算完了
0.4385383129119873:モザイクアート生成開始
残りタイル：20


IndexError: ignored